In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
pvar_f <- '/oak/stanford/groups/mrivas/ukbb24983/array-combined/pgen/ukb24983_cal_hla_cnv.pvar'
data_d <- '/oak/stanford/groups/mrivas/ukbb24983/array-combined/annotation/afreq_20201012'
snp_qc_f <- '/oak/stanford/groups/mrivas/ukbb24983/snp/ukb_snp_qc.pvar.zst'
out_f <- file.path(data_d, 'ukb24983_cal_hla_cnv.afreq_20201012.pvar')

In [48]:
pops <- c('white_british', 'non_british_white', 'african', 's_asian', 'e_asian', 'related', 'others')
arrays <- c('UKBB', 'UKBL')


In [4]:
cat_or_zcat <- function(f){
    ifelse(endsWith(f, '.zst'), 'zstdcat', ifelse(endsWith(f, '.gz'), 'zcat', 'cat'))
}


In [5]:
read_tbl <- function(pops, prefix, suffix){
    pops %>% 
    lapply(function(pop){
        if(pop == 'all'){
            f <- sprintf('%s.%s', prefix, suffix)
        }else{
            f <- sprintf('%s.%s.%s', prefix, pop, suffix)
        }
        fread(
            cmd=paste(cat_or_zcat(f), f), 
            colClasses=c('#CHROM'='character')
        ) %>%
        rename('CHROM'='#CHROM') %>%
        mutate(population = pop)
    }) %>% bind_rows()
}


In [6]:
spread_pop <- function(df, col, pops, col_prefix, col_suffix, col_sep='_'){    
    df %>% select(ID, col, population) %>%
    rename(!!'VAL' := col) %>% spread(population, VAL) %>%
    select(ID, all_of(pops)) %>% rename(setNames(
        1:length(pops)+1, 
        paste(rep(col_prefix, length(pops)), pops, rep(col_suffix, length(pops)), sep=col_sep)
    ))
}


In [19]:
fread(cmd=paste('zstdcat', snp_qc_f), select=c('ID', 'array'), colClasses = 'character') %>%
rename('array_code'='array') %>%
mutate(
    # 0=BiLEVE, 1=Axiom, 2=both
    # see: http://biobank.ctsu.ox.ac.uk/crystal/refer.cgi?id=1955
    array = if_else(array_code == '0', 'UKBL', if_else(array_code == '1', 'UKBB', 'both'))
)-> snp_qc_df


In [20]:
snp_qc_df %>% count(array, array_code)


array,array_code,n
<chr>,<chr>,<int>
both,2,753693
UKBB,1,34197
UKBL,0,17536


In [18]:
pvar_f %>%
fread(colClasses = c('#CHROM'='character')) %>%
rename('CHROM'='#CHROM') -> pvar_df


In [9]:
read_tbl(
    c('all', pops), file.path(data_d, 'plink_output', 'ukb24983_cal_hla_cnv'), 'afreq.zst'
) -> afreq_long_df


In [49]:
read_tbl(
    c('all', pops, arrays), file.path(data_d, 'plink_output', 'ukb24983_cal_hla_cnv'), 'gcount.zst'
) -> gcount_long_df


In [50]:
pvar_df %>%
left_join(snp_qc_df %>% select(-array_code), by='ID') %>%
left_join(
    afreq_long_df %>%
    spread_pop('ALT_FREQS', c('all', pops), 'UKB', 'AF') %>%
    rename('UKB_AF'=2),
    by='ID'
) -> pvar_af_df


In [51]:
for(col in c('OBS_CT', 'MISSING_CT')){
    message(col)
    pvar_af_df %>%
    left_join(
        gcount_long_df %>%
        spread_pop(col, c('all', pops, arrays), 'UKB', col) %>%
        rename(!!sprintf('UKB_%s', col) := 2),
        by='ID'
    ) -> pvar_af_df
}

OBS_CT

MISSING_CT



In [52]:
for(col in c('HOM_REF_CT', 'HET_REF_ALT_CTS', 'TWO_ALT_GENO_CTS', 'HAP_REF_CT', 'HAP_ALT_CTS')){
    message(col)
    pvar_af_df %>%
    left_join(
        gcount_long_df %>%
        spread_pop(col, c('all', pops), 'UKB', col) %>%
        rename(!!sprintf('UKB_%s', col) := 2),
        by='ID'
    ) -> pvar_af_df
}

HOM_REF_CT

HET_REF_ALT_CTS

TWO_ALT_GENO_CTS

HAP_REF_CT

HAP_ALT_CTS



In [54]:
pvar_af_df %>% 
mutate(
    f_miss = UKB_MISSING_CT / (UKB_MISSING_CT + UKB_OBS_CT),
    f_miss_UKBB = if_else((!is.na(array)) & array == 'UKBL', as.double(NA), UKB_UKBB_MISSING_CT / (UKB_UKBB_MISSING_CT + UKB_UKBB_OBS_CT)),
    f_miss_UKBL = if_else((!is.na(array)) & array == 'UKBB', as.double(NA), UKB_UKBL_MISSING_CT / (UKB_UKBL_MISSING_CT + UKB_UKBL_OBS_CT))
) -> pvar_af_df


In [59]:
pvar_af_df %>% dim()

[1] 1080968      77

In [60]:
pvar_af_df %>% colnames()

[1] "CHROM"                                 
 [2] "POS"                                   
 [3] "ID"                                    
 [4] "REF"                                   
 [5] "ALT"                                   
 [6] "array"                                 
 [7] "UKB_AF"                                
 [8] "UKB_white_british_AF"                  
 [9] "UKB_non_british_white_AF"              
[10] "UKB_african_AF"                        
[11] "UKB_s_asian_AF"                        
[12] "UKB_e_asian_AF"                        
[13] "UKB_related_AF"                        
[14] "UKB_others_AF"                         
[15] "UKB_OBS_CT"                            
[16] "UKB_white_british_OBS_CT"              
[17] "UKB_non_british_white_OBS_CT"          
[18] "UKB_african_OBS_CT"                    
[19] "UKB_s_asian_OBS_CT"                    
[20] "UKB_e_asian_OBS_CT"                    
[21] "UKB_related_OBS_CT"                    
[22] "UKB_others_OBS_CT"                     
[23] "UKB_UKBB_OBS_CT"                       
[24] "UKB_UKBL_OBS_CT"                       
[25] "UKB_MISSING_CT"                        
[26] "UKB_white_british_MISSING_CT"          
[27] "UKB_non_british_white_MISSING_CT"      
[28] "UKB_african_MISSING_CT"                
[29] "UKB_s_asian_MISSING_CT"                
[30] "UKB_e_asian_MISSING_CT"                
[31] "UKB_related_MISSING_CT"                
[32] "UKB_others_MISSING_CT"                 
[33] "UKB_UKBB_MISSING_CT"                   
[34] "UKB_UKBL_MISSING_CT"                   
[35] "UKB_HOM_REF_CT"                        
[36] "UKB_white_british_HOM_REF_CT"          
[37] "UKB_non_british_white_HOM_REF_CT"      
[38] "UKB_african_HOM_REF_CT"                
[39] "UKB_s_asian_HOM_REF_CT"                
[40] "UKB_e_asian_HOM_REF_CT"                
[41] "UKB_related_HOM_REF_CT"                
[42] "UKB_others_HOM_REF_CT"                 
[43] "UKB_HET_REF_ALT_CTS"                   
[44] "UKB_white_british_HET_REF_ALT_CTS"     
[45] "UKB_non_british_white_HET_REF_ALT_CTS" 
[46] "UKB_african_HET_REF_ALT_CTS"           
[47] "UKB_s_asian_HET_REF_ALT_CTS"           
[48] "UKB_e_asian_HET_REF_ALT_CTS"           
[49] "UKB_related_HET_REF_ALT_CTS"           
[50] "UKB_others_HET_REF_ALT_CTS"            
[51] "UKB_TWO_ALT_GENO_CTS"                  
[52] "UKB_white_british_TWO_ALT_GENO_CTS"    
[53] "UKB_non_british_white_TWO_ALT_GENO_CTS"
[54] "UKB_african_TWO_ALT_GENO_CTS"          
[55] "UKB_s_asian_TWO_ALT_GENO_CTS"          
[56] "UKB_e_asian_TWO_ALT_GENO_CTS"          
[57] "UKB_related_TWO_ALT_GENO_CTS"          
[58] "UKB_others_TWO_ALT_GENO_CTS"           
[59] "UKB_HAP_REF_CT"                        
[60] "UKB_white_british_HAP_REF_CT"          
[61] "UKB_non_british_white_HAP_REF_CT"      
[62] "UKB_african_HAP_REF_CT"                
[63] "UKB_s_asian_HAP_REF_CT"                
[64] "UKB_e_asian_HAP_REF_CT"                
[65] "UKB_related_HAP_REF_CT"                
[66] "UKB_others_HAP_REF_CT"                 
[67] "UKB_HAP_ALT_CTS"                       
[68] "UKB_white_british_HAP_ALT_CTS"         
[69] "UKB_non_british_white_HAP_ALT_CTS"     
[70] "UKB_african_HAP_ALT_CTS"               
[71] "UKB_s_asian_HAP_ALT_CTS"               
[72] "UKB_e_asian_HAP_ALT_CTS"               
[73] "UKB_related_HAP_ALT_CTS"               
[74] "UKB_others_HAP_ALT_CTS"                
[75] "f_miss"                                
[76] "f_miss_UKBB"                           
[77] "f_miss_UKBL"

In [61]:
pvar_af_df %>%
rename('#CHROM' = 'CHROM') %>%
fwrite(out_f, sep='\t', na = "NA", quote=F)


In [62]:
out_f

[1] "/oak/stanford/groups/mrivas/ukbb24983/array-combined/annotation/afreq_20201012/ukb24983_cal_hla_cnv.afreq_20201012.pvar"